<a href="https://colab.research.google.com/github/Baah134/Baah134/blob/main/Titanic_Challenge/Titanic_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/DeepLearning/Titanic_Challenge/train.csv'

df = pd.read_csv(path)
df.sample(10)

In [ ]:
df.shape

In [ ]:
#Dropping data that is either incomplete or irrelevant
df.drop('Name', axis='columns',inplace=True)
df.drop('Cabin', axis='columns',inplace=True)
df.drop('Ticket', axis='columns',inplace=True)
#df.drop('PassengerId', axis='columns',inplace=True)

In [ ]:
df.shape

In [ ]:
df.iloc[630].Age

In [ ]:
df.dropna(subset=['Age'], inplace=True)
df.dropna(subset=['Embarked'], inplace=True)

In [ ]:
df.shape

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
def print_unique_col_values(df):
       for column in df:
            if df[column].dtypes=='object':
                print(f'{column}: {df[column].unique()}')

In [ ]:
print_unique_col_values(df)

In [ ]:
df['Sex'].replace({'female':1,'male':0},inplace=True)

In [ ]:
for col in df:
    print(f'{col}: {df[col].unique()}')

In [ ]:
#One-hot encoding for Categorical Columns
df = pd.get_dummies(data=df, columns=['Embarked'], dtype= int)
df.columns

In [ ]:
df.dtypes

In [ ]:
cols_to_scale = ['Age','Fare','SibSp','Parch','Pclass']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

In [ ]:
df.sample(10)

In [ ]:
X = df.drop('Survived',axis='columns')
y = df['Survived']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=5)

In [ ]:
X_train.shape

In [ ]:
X_train[:10]

In [ ]:
import tensorflow as tf
from tensorflow import keras


model = keras.Sequential([
    keras.layers.Dense(10, input_shape=(10,), activation='relu'),
    keras.layers.Dense(25, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(90, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.save_weights('my_model_weights.weights.h5')


In [ ]:
new_data_path = '/content/drive/MyDrive/DeepLearning/Titanic_Challenge/test.csv'  # Replace with the actual path
new_df = pd.read_csv(new_data_path)

In [ ]:
new_df.sample(10)

In [ ]:
#Dropping data that is either incomplete or irrelevant
new_df.drop('Name', axis='columns',inplace=True)
new_df.drop('Cabin', axis='columns',inplace=True)
new_df.drop('Ticket', axis='columns',inplace=True)


new_df.fillna(0, inplace=True)

new_df['Sex'].replace({'female':1,'male':0},inplace=True)



cols_to_scale = ['Age','Fare','SibSp','Parch','Pclass',"PassengerId"]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
new_df[cols_to_scale] = scaler.fit_transform(new_df[cols_to_scale])

In [ ]:
new_df.dtypes

In [ ]:
#One-hot encoding for Categorical Columns
new_df = pd.get_dummies(data=new_df, columns=['Embarked'], dtype= int)
new_df.columns

In [ ]:
new_df.sample(10)

In [ ]:
model.load_weights('my_model_weights.weights.h5')
predictions = model.predict(new_df)
predicted_labels = (predictions > 0.5).astype(int)


In [ ]:
nnew_df = pd.read_csv(new_data_path)

In [ ]:
predictions_df = pd.DataFrame({'PassengerId': nnew_df['PassengerId'], 'Survived': predicted_labels.ravel()}) #Flatten predicted_labels to 1D
predictions_df.to_csv('predictions.csv', index=False)  # Set index=False to avoid saving the index
from google.colab import files
files.download('predictions.csv')